In [71]:
import sys, os
import numpy as np
import tensorflow as tf
from datetime import datetime

#Local methods:
from alexnet import AlexNet
from data_loader import data_loader

sys.path.append('../')
from generator import Generator

In [72]:
tf.reset_default_graph() 

In [73]:
#INIT 1
mode = 'train'
'''
Model class for you to modify for brick/ball/cylinder deep learning challenge.
mode = 'train' or 'test' - could be a useful flag if we want to use dataset augmentations. 
'''
if mode == 'train' or mode == 'test':
     mode = mode
else:
    print('Mode must be train or test.')
print("In INit")
# Learning params
learning_rate = 1e-3
minibatch_size = 32
num_iterations = 1500
train_test_split = 0.7

# Network params
keep_rate = 0.5 #Fraction of connections that we keep when performing dropout

# How many pretrained alexnet layers to we want to load up?
# This must be a number between 1 and 8. 
num_layers_to_load = 7

# How often we want to write the tf.summary data to disk and measure performance
display_step = 5
save_step = 100

# Path for tf.summary.FileWriter and to store model checkpoints
save_dir = "tf_data/sample_model"

input_image_size = (227, 227)
label_indices = {'brick': 0, 'ball': 1, 'cylinder': 2}
labels_ordered = list( label_indices)
num_classes = len( label_indices)

channel_means = np.array([147.12697, 160.21092, 167.70029])

#Go ahead and build graph on initialization:
#build_graph()

#Set to true with we create a tf session.
initialized = False

In INit


In [74]:
############ def build_graph(self):
'''
This is where you can experiment with various architectures. 
'''
print("IN BUILD")
# TF placeholder for graph input and output
X = tf.placeholder(tf.float32, [None, 
                                input_image_size[0], 
                                input_image_size[1], 
                                3])

y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32)

# Build Alexnet portion of graph
AN = AlexNet(X, keep_prob, num_layers_to_load)

#### ------------ Setup Your Graph Here ------------------ ####

# We're taking the front of our graph from AlexNet, now let's 
# build the rest - this is where we get to be creative!
# Grab the tensor from alexnet that we're going to build from
# Be sure to change this if you chnage num_layers_to_load
# alexnet_out = AN.pool5
alexnet_out = AN.fc7

#To see the dimension of the output we're getting from alexnet, we can print our tensor:
print(alexnet_out)

def fc(x, num_in, num_out, name, relu=True):
    """Create a fully connected layer."""
    with tf.variable_scope(name) as scope:

        # Create tf variables for the weights and biases
        weights = tf.get_variable('weights', shape=[num_in, num_out],
                                  trainable=True)
        biases = tf.get_variable('biases', [num_out], trainable=True)

        # Matrix multiply weights and inputs and add bias
        act = tf.nn.xw_plus_b(x, weights, biases, name=scope.name)

    if relu:
        # Apply ReLu non linearity
        relu = tf.nn.relu(act)
        return relu
    else:
        return act
fc8 = fc(AN.fc7, 4096, 3, relu=False, name='fc8')
# all_variable_names.extend(['fc8'])




# Your graph should produce an estimate our our labels, yhat.
# yhat should be of the same dimension as y.
# self.yhat = ?
logits = tf.layers.dense(inputs = fc8, units = 3, activation = None, name = 'fc9')
yhat = tf.nn.softmax(logits)

# Cross Entropy
# Implmenting cross entropy "manually" in tensoflow can be numerically unstable, so use this method:




# Make sure you inlude the names of all the variables you would like to train here. 
# These may include variables from this part of the graph or the alexnet portion. 
# You can give your variables whatever name you like by passing in a name into your layers
# for example if you setup a layer like this: tf.layers.dense(.... , name = 'my_fc6')
# be sure to add 'my_fc6' to the trainable_variable_names list here:
trainable_variable_names = ['fc8']

#### ---------------- End Graph Setup --------------------- ####

IN BUILD
Tensor("Relu_1:0", shape=(?, 4096), dtype=float32)


In [75]:
#def train(self):
'''
Train model.
'''
print("In TRAIN")
# Start by building rest of graph that we needed for training.

#### ------------ Setup Cost Function Here ------------------ ####

# Your job here is to compute a cost to pass into our AdamOptimizer below. 
# cost = 


cost = tf.losses.softmax_cross_entropy(y, logits)


#### ------------ End Cost Function Setup  ------------------ ####

# We only want to train some of the variables in our overall graph:
var_list = [v for v in tf.trainable_variables() if v.name.split('/')[0] \
                                            in  trainable_variable_names]

# Train op
with tf.name_scope("train"):
    # Get gradients of all trainable variables
    gradients = tf.gradients(cost, var_list)
    gradients = list(zip(gradients, var_list))

    # Create optimizer and apply gradient descent to the trainable variables
    optimizer = tf.train.AdamOptimizer( learning_rate)
    train_op = optimizer.apply_gradients(grads_and_vars=gradients)

# Add gradients to summary
for gradient, var in gradients:
    tf.summary.histogram(var.name + '/gradient', gradient)

# Add the variables we train to the summary
for var in var_list:
    tf.summary.histogram(var.name, var)

# Add the loss to summary
tf.summary.scalar('cross_entropy', cost)

# Evaluation op: Accuracy of the model
with tf.name_scope("accuracy"):
    correct_pred = tf.equal(tf.argmax( yhat, 1), tf.argmax( y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Add the accuracy to the summary
tf.summary.scalar('accuracy', accuracy)

# Merge all summaries together
merged_summary = tf.summary.merge_all()

# Initialize the FileWriter
train_writer = tf.summary.FileWriter(os.path.join( save_dir, 'train'))
test_writer = tf.summary.FileWriter(os.path.join( save_dir, 'test'))

# Initialize an saver for store model checkpoints
# We'll save the pretrained alexnet weights and the new weight values that we train:
var_list_to_save = [v for v in tf.trainable_variables() if v.name.split('/')[0] in \
                                 AN.all_variable_names +  trainable_variable_names]

saver = tf.train.Saver(var_list = var_list_to_save)


#Load up data from image files. 
data = data_loader(label_indices =  label_indices, 
           channel_means =  channel_means, 
           train_test_split =  train_test_split,
           input_image_size =  input_image_size, 
           data_path = '../data')

#Setup minibatch generators
G = Generator(data.train.X, data.train.y, minibatch_size =  minibatch_size)
GT = Generator(data.test.X, data.test.y, minibatch_size =  minibatch_size)

#Launch tf session
sess = tf.Session()
sess.run(tf.global_variables_initializer())
initialized = True

# Add the model graph to TensorBoard
train_writer.add_graph( sess.graph)

# Load the pretrained weights into the alexnet portion of our graph
AN.load_initial_weights( sess)

print("{} Start training...".format(datetime.now()))
print("{} Open Tensorboard at --logdir {}".format(datetime.now(),
                                                   save_dir))


#And Train
for i in range( num_iterations):
    G.generate()
    # And run the training op
    sess.run(train_op, feed_dict={ X: G.X,  y: G.y,  keep_prob:  keep_rate})


    # Generate summary with the current batch of data and write to file
    if i %  display_step == 0:
        s =  sess.run(merged_summary, feed_dict={ X: G.X,  y: G.y,  keep_prob: 1.0})
        train_writer.add_summary(s, i)

        GT.generate()
        s =  sess.run(merged_summary, feed_dict={ X: GT.X,  y: GT.y,  keep_prob: 1.0})
        test_writer.add_summary(s, i)

        train_acc =  sess.run(accuracy, feed_dict={ X: G.X,  y: G.y,  keep_prob: 1.0})
        test_acc =  sess.run(accuracy, feed_dict={ X: GT.X,  y: GT.y,  keep_prob: 1.0})

        print(i, ' iterations,', str(G.num_epochs), 'epochs, train accuracy = ', train_acc, ', test accuracy = ', test_acc)

    if i %  save_step == 0 and i > 0: 
        print("{} Saving checkpoint of model...".format(datetime.now()))

        # save checkpoint of the model
        checkpoint_name = os.path.join(save_dir,
                                       'model_epoch'+str(G.num_epochs)+'.ckpt')
        save_path = saver.save(sess, checkpoint_name)

        print("{} Model checkpoint saved at {}".format(datetime.now(),
                                                       checkpoint_name))

In TRAIN
INFO:tensorflow:Summary name fc8/weights:0/gradient is illegal; using fc8/weights_0/gradient instead.
INFO:tensorflow:Summary name fc8/biases:0/gradient is illegal; using fc8/biases_0/gradient instead.
INFO:tensorflow:Summary name fc8/weights:0 is illegal; using fc8/weights_0 instead.
INFO:tensorflow:Summary name fc8/biases:0 is illegal; using fc8/biases_0 instead.


IndexError: list index out of range

In [7]:
#def predict(self, X, checkpoint_dir = None):
'''
X: Numpy array of dimension [n, 227, 227, 3].
Note that n here may not be the same as the minibatch_size defined above.

Returns:
yhat_numpy: A numpy array of dimension [n x 3] containing the predicted 
one hot labels for each image passed in X. 
'''
if not initialized:
    restore_from_checkpoint(checkpoint_dir)

yhat_numpy = sess.run(yhat, feed_dict = {X : X, keep_prob: 1.0})

return yhat_numpy

In [8]:
#def restore_from_checkpoint(self, checkpoint_dir = None):
'''
Restore model from most recent checkpoint in save dir.
'''

saver = tf.train.Saver()
sess = tf.Session()

#Load latest checkpont, use savedir if we're not given a checkpoint dir:
if checkpoint_dir is None:
    checkpoint_name = tf.train.latest_checkpoint(save_dir)
else:
    checkpoint_name = tf.train.latest_checkpoint(checkpoint_dir)

print(checkpoint_name)

#Resore session
saver.restore(sess, checkpoint_name)
initialized = True

In INit
IN BUILD
Tensor("pool5:0", shape=(?, 6, 6, 256), dtype=float32)
